# Sample CloudSql Connection

**NOTE**: This will only work within the GCP notebook instance with the current permissions that our account has on secretmanager settings.

In [1]:
from google.cloud import secretmanager
import pandas as pd
from sqlalchemy import create_engine, text

#### Configuration

In [3]:
PROJECT_ID = 'adsp-34002-on02-sopho-scribe'
REGION = 'us-central1'
DB_NAME = 'postgres'
DB_HOST = '35.232.92.211'
DB_PORT = '5432'

#### Retrieve credentials from Secret Manager

In [4]:
def access_secret(secret_id):
    client = secretmanager.SecretManagerServiceClient()
    name = f"projects/{PROJECT_ID}/secrets/{secret_id}/versions/latest"
    response = client.access_secret_version(name=name)
    return response.payload.data.decode("UTF-8")

In [ ]:
DB_USER = access_secret('cloudSqlUser')
DB_PASSWORD = access_secret('cloudSqlUserPassword')

#### Create SQLAlchemy engine

In [5]:
engine = create_engine(
    f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}',
    connect_args={'sslmode': 'require'}
)

#### Generate & insert sample data

In [6]:
# Create sample data
sample_data = pd.DataFrame({
    'name': ['Alice', 'Bob', 'Charlie'],
    'score': [88, 92, 85]
})

# Create table and insert data
with engine.connect() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS test_scores (
            id SERIAL PRIMARY KEY,
            name VARCHAR(100),
            score INT
        )
    """))

    for _, row in sample_data.iterrows():
        conn.execute(
            text("INSERT INTO test_scores (name, score) VALUES (:name, :score)"),
            {"name": row['name'], "score": row['score']}
        )

#### Read back the data

In [7]:
df_result = pd.read_sql("SELECT * FROM test_scores", con=engine)
print(df_result)

   id     name  score
0   1    Alice     88
1   2      Bob     92
2   3  Charlie     85


In [8]:
import datetime
import pytz

print(f"Notebook last execution time: {datetime.datetime.now(pytz.timezone('US/Central')).strftime('%a, %d %B %Y %H:%M:%S')}")

Notebook last execution time: Mon, 21 April 2025 21:54:44
